In [1]:
!python --version

Python 3.12.2


In [10]:
#importing necessary modules 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import seaborn as sns
import copairs
import pycytominer
from copairs.map import average_precision
from copairs.map import mean_average_precision
from utils import * 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
pip show copairs

Name: copairs
Version: 0.4.1
Summary: Find pairs and compute metrics between them
Home-page: 
Author: 
Author-email: John Arevalo <johnarevalo@gmail.com>
License: 
Location: c:\Users\ssivagur\Anaconda3\envs\copairs\Lib\site-packages
Requires: pandas, statsmodels, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


### Phenotypic activity

#### Reading the result csv of copairs to get add the cell count info 

In [4]:
result_csv ={'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122250_normalized_feature_select_negcon_batch_Result_NegconNorm_mAP_wrt_Control_woNAN.csv':'Standard CP', 
             'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122246_normalized_feature_select_negcon_batch_Result_NegconNorm_mAP_wrt_Control_woNAN.csv':'CP + MitoBrilliant',
            'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122247_normalized_feature_select_negcon_batch_Result_NegconNorm_mAP_wrt_Control_woNAN.csv':'CP + Phenovue phalloidin 400LS',
            'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122248_normalized_feature_select_negcon_batch_Result_NegconNorm_mAP_wrt_Control_woNAN.csv':'Standard CP (exposed to ChromaLive)',
            'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122249_normalized_feature_select_negcon_batch_wo_phasefeatures_Result_NegconNorm_mAP_wrt_Control_woNAN.csv':'ChromaLive + Hoechst'}

#### Reading the aggregate csv files of copairs 

In [5]:
aggregate_files_dict = {'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122250_normalized_feature_select_negcon_batch_Aggregate_result_NegconNorm_mAP_wrt_Control_woNAN.csv':'Standard CP',
                        'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122246_normalized_feature_select_negcon_batch_Aggregate_result_NegconNorm_mAP_wrt_Control_woNAN.csv':'CP + MitoBrilliant',
                        'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122247_normalized_feature_select_negcon_batch_Aggregate_result_NegconNorm_mAP_wrt_Control_woNAN.csv':'CP + Phenovue phalloidin 400LS',
                        'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122248_normalized_feature_select_negcon_batch_Aggregate_result_NegconNorm_mAP_wrt_Control_woNAN.csv':'Standard CP (exposed to ChromaLive)',
                        'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122249_normalized_feature_select_negcon_batch_wo_phasefeatures_Aggregate_result_NegconNorm_mAP_wrt_Control_woNAN.csv':'ChromaLive + Hoechst'}

In [6]:
moA_control_plot, df_controls = cell_count_norm_colorscheme(result_csv, aggregate_files_dict)

### Plot

In [7]:
scatter_plot_comparison_colorblack(df_controls,col1='Standard CP', col2='CP + MitoBrilliant', cutoff=0.3)

In [8]:
scatter_plot_comparison_colorblack(df_controls,col1='Standard CP', col2='CP + Phenovue phalloidin 400LS', cutoff=0.35)

In [9]:
scatter_plot_comparison_colorblack(df_controls,col1='Standard CP', col2='ChromaLive + Hoechst', cutoff=0.55)

### Plot with trendline

In [11]:
scatter_plot_comparison_colorblack_trendline(df_controls,col1='Standard CP', col2='CP + MitoBrilliant')

In [12]:
scatter_plot_comparison_colorblack_trendline(df_controls,col1='Standard CP', col2='CP + Phenovue phalloidin 400LS')

In [13]:
scatter_plot_comparison_colorblack_trendline(df_controls,col1='Standard CP', col2='ChromaLive + Hoechst')

### Phenotypic distinctiveness

#### mAP with respect to treatments - MoA

In [14]:
#Reading the result CSVs obtained from the copairs 
result_csv_trmts = {'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122250_normalized_feature_select_negcon_batch_Result_NegconNorm_mAP_wrt_Treatment_woNAN.csv':'Standard CP',
                    'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122246_normalized_feature_select_negcon_batch_Result_NegconNorm_mAP_wrt_Treatment_woNAN.csv':'CP + MitoBrilliant',
                    'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122247_normalized_feature_select_negcon_batch_Result_NegconNorm_mAP_wrt_Treatment_woNAN.csv':'CP + Phenovue phalloidin 400LS',
                    'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122248_normalized_feature_select_negcon_batch_Result_NegconNorm_mAP_wrt_Treatment_woNAN.csv':'Standard CP (exposed to ChromaLive)',
                    'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122249_normalized_feature_select_negcon_batch_wo_phasefeatures_Result_NegconNorm_mAP_wrt_Treatment_woNAN.csv':'ChromaLive + Hoechst'}

In [15]:
#Reading the aggregate CSV files obtained from the copairs package
aggregate_files_trmt = {'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122250_normalized_feature_select_negcon_batch_Aggregate_result_NegconNorm_mAP_wrt_Treatment_woNAN.csv':'Standard CP',
                    'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122246_normalized_feature_select_negcon_batch_Aggregate_result_NegconNorm_mAP_wrt_Treatment_woNAN.csv':'CP + MitoBrilliant',
                    'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122247_normalized_feature_select_negcon_batch_Aggregate_result_NegconNorm_mAP_wrt_Treatment_woNAN.csv':'CP + Phenovue phalloidin 400LS',
                    'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122248_normalized_feature_select_negcon_batch_Aggregate_result_NegconNorm_mAP_wrt_Treatment_woNAN.csv':'Standard CP (exposed to ChromaLive)',
                    'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122249_normalized_feature_select_negcon_batch_wo_phasefeatures_Aggregate_result_NegconNorm_mAP_wrt_Treatment_woNAN.csv':'ChromaLive + Hoechst'}

In [16]:
mAP_treatment_plot_colorscheme, df_treatment = cell_count_norm_colorscheme(result_csv_trmts, aggregate_files_trmt)

### Plot

In [17]:
scatter_plot_comparison_colorblack(df_treatment, col1='Standard CP', col2='CP + MitoBrilliant', cutoff=0.4)

In [18]:
scatter_plot_comparison_colorblack(df_treatment, col1='Standard CP', col2='CP + Phenovue phalloidin 400LS', cutoff=0.39)

In [19]:
scatter_plot_comparison_colorblack(df_treatment, col1='Standard CP', col2='ChromaLive + Hoechst', cutoff=0.71)


### Plot with trendline

In [20]:
scatter_plot_comparison_colorblack_trendline(df_treatment, col1='Standard CP', col2='CP + MitoBrilliant')

In [21]:
scatter_plot_comparison_colorblack_trendline(df_treatment, col1='Standard CP', col2='CP + Phenovue phalloidin 400LS')

In [22]:
scatter_plot_comparison_colorblack_trendline(df_treatment, col1='Standard CP', col2='ChromaLive + Hoechst')